In [5]:
import pandas as pd
import numpy as np

INITIAL_CAPITAL = 100000

In [6]:
def load_data(file_path: str = '../data/trade_history.csv') -> pd.DataFrame:
    # Load the CSV into a DataFrame
    df = pd.read_csv(file_path)
    
    # Convert to datetime
    date_columns = ['Entry time', 'Exit time']
    df[date_columns] = df[date_columns].apply(pd.to_datetime)
    
    # Convert percentage return and drawdown to float
    percent_columns = ['Return', 'Drawdown']
    df[percent_columns] = df[percent_columns].apply(lambda col: col.str.rstrip('%').astype(float)) / 100
    
    # Remove dollar sign and commas, convert to float for specified columns
    dollar_columns = ['Profit', 'Cum. net profit', 'Cum. cash value', 'Commission']
    df[dollar_columns] = df[dollar_columns].apply(lambda col: col.str.replace('$', '').str.replace(',', '').astype(float))

    return df

df = load_data()
df.head()

,Trade number,Instrument,Account,Strategy,Market pos.,Qty,Entry price,Exit price,Entry time,Exit time,...,ETD,Bars,HWM,Drawdown,Return,Instrument Vol,Instrument Vol Floor,Annual Vol Target,Annual Realized Vol,Strat Scalar
0,1,NQ,Backtest,TrendVol,Long,4,5020.50,5018.50,2017-01-10 02:54:00,2017-01-10 09:40:00,...,$782.72,15,"$100,000.00",-0.0018,-0.0018,0.000819,0.002,0.2,0.166956,0.778614
1,2,NQ,Backtest,TrendVol,Long,3,5028.50,5036.25,2017-01-10 10:56:00,2017-01-10 13:51:00,...,$662.04,7,"$100,265.24",0.0000,0.0045,0.000775,0.002,0.2,0.178878,0.651149
2,3,NQ,Backtest,TrendVol,Long,3,5047.50,5041.50,2017-01-11 16:03:00,2017-01-11 19:14:00,...,$422.04,5,"$100,265.24",-0.0038,-0.0038,0.000754,0.002,0.2,0.155804,0.632381
3,4,NQ,Backtest,TrendVol,Long,4,5048.75,5053.50,2017-01-13 09:34:00,2017-01-13 12:28:00,...,$802.72,6,"$100,265.24",-0.0002,0.0036,0.000751,0.002,0.2,0.118007,1.236208
4,5,NQ,Backtest,TrendVol,Long,4,5054.50,5048.25,2017-01-18 10:05:00,2017-01-18 10:09:00,...,$522.72,1,"$100,265.24",-0.0054,-0.0052,0.000757,0.002,0.2,0.098349,1.859764


In [7]:
# Group daily aggregates
daily = df.groupby(df['Entry time'].dt.date).agg({
    'Return': 'sum',
    'Drawdown': 'min'
}).reset_index(drop=True)

In [8]:
# Calculate values
duration_years = (df['Exit time'].max() - df['Entry time'].min()).days / 365.25
net_profit = df['Cum. net profit'].iloc[-1] - df['Cum. net profit'].iloc[0]
total_commission = df['Commission'].sum()
cagr = (((df['Cum. cash value'].iloc[-1] / INITIAL_CAPITAL) ** (1 / duration_years)) - 1)
mean_return = daily['Return'].mean()
annualized_volatility = daily['Return'].std() * np.sqrt(253)
daily_volatility = daily['Return'].std()
sharpe_ratio = (daily['Return'].mean() / daily['Return'].std()) * np.sqrt(253)
sortino_ratio = daily['Return'].mean() / daily[daily['Return'] < 0]['Return'].std() * np.sqrt(253)
win_rate = len(df[df['Profit'] > 0]) / len(df)
average_hold_time = ((df['Exit time'] - df['Entry time']).dt.total_seconds() / 60).mean()
max_drawdown = daily['Drawdown'].min()

trading_metrics = pd.Series({
    'Start Date': df['Entry time'].min().strftime('%Y-%m-%d'),
    'End Date': df['Exit time'].max().strftime('%Y-%m-%d'),
    'Initial Capital': f"${INITIAL_CAPITAL:,.2f}",
    'Net Profit': f"${net_profit:,.2f}",
    'Total Commission': f"${total_commission:,.2f}",
    'Total Trades': f"{len(df):,}",
    'CAGR': f"{cagr:.2%}",
    'Mean Return': f"{mean_return:.2%}",
    'Daily Volatility': f"{daily_volatility:.2%}",
    'Annualized Volatility': f"{annualized_volatility:.2%}",
    'Sharpe Ratio': f"{sharpe_ratio:.2f}",
    'Sortino Ratio': f"{sortino_ratio:.2f}",
    'Max Drawdown': f"{max_drawdown:.2%}",
    'Win Rate': f"{win_rate:.2%}",
    'Average Hold Time': f"{average_hold_time:.0f} min",
    'Kurtosis': f"{daily['Return'].kurtosis():.2f}",
    'Skewness': f"{daily['Return'].skew():.2f}"
})

trading_metrics_df = trading_metrics.to_frame(name='Value')

print("\nTrendVol Performance at 20% Vol")
print("=" * 40)
print(trading_metrics_df)
print("\nNote: Market is NQ futures, stats based on $100,000 starting capital.")
print("Trading commissions and 2 tick slippage included ~ 1809 total slippage.")
print("20% annualized volatility as the target.")


TrendVol Performance at 20% Vol
                               Value
Start Date                2017-01-10
End Date                  2024-12-26
Initial Capital          $100,000.00
Net Profit             $1,628,023.36
Total Commission          $46,729.36
Total Trades                   1,809
CAGR                          43.05%
Mean Return                    0.26%
Daily Volatility               1.33%
Annualized Volatility         21.22%
Sharpe Ratio                    3.09
Sortino Ratio                  10.19
Max Drawdown                  -7.85%
Win Rate                      36.82%
Average Hold Time            584 min
Kurtosis                       10.27
Skewness                        2.55

Note: Market is NQ futures, stats based on $100,000 starting capital.
Trading commissions and 2 tick slippage included ~ 1809 total slippage.
20% annualized volatility as the target.
